# Joins in `pyspark`

Performed with `df_left.join(df_right, how=type_str)`

In [1]:
from pyspark.sql import SparkSession
from more_pyspark import to_pandas
spark = SparkSession.builder.appName('Ops').getOrCreate()
deptk = spark.read.csv("./data/department.csv",  header=True, inferSchema=True)
deptk.collect() >> to_pandas

,DeptID,DeptName
0,31,Sales
1,33,Engineering
2,34,Clerical
3,35,Marketing


In [2]:
emplk = spark.read.csv("./data/employee.csv",  header=True, inferSchema=True)
emplk.collect() >> to_pandas

,DeptID,LastName
0,31.0,Rafferty
1,33.0,Jones
2,33.0,Heisenberg
3,34.0,Robinson
4,34.0,Smith
5,NaN,Williams


#### Inner join

In [3]:
(emplk.join(deptk, emplk.DeptID == deptk.DeptID, how='inner')
 .collect()) >> to_pandas

,DeptID,LastName,DeptName
0,31,Rafferty,Sales
1,33,Jones,Engineering
2,33,Heisenberg,Engineering
3,34,Robinson,Clerical
4,34,Smith,Clerical


#### Left join

In [4]:
(emplk.join(deptk, emplk.DeptID == deptk.DeptID, how='left')
 .collect()) >> to_pandas

,DeptID,LastName,DeptName
0,31.0,Rafferty,Sales
1,33.0,Jones,Engineering
2,33.0,Heisenberg,Engineering
3,34.0,Robinson,Clerical
4,34.0,Smith,Clerical
5,NaN,Williams,None


#### Right join

In [5]:
(emplk.join(deptk, emplk.DeptID == deptk.DeptID, how='right')
 .collect()) >> to_pandas

,DeptID,LastName,DeptName
0,31,Rafferty,Sales
1,33,Heisenberg,Engineering
2,33,Jones,Engineering
3,34,Smith,Clerical
4,34,Robinson,Clerical
5,35,None,Marketing


#### Outer join

In [6]:
(emplk.join(deptk, emplk.DeptID == deptk.DeptID, how='outer')
 .collect()) >> to_pandas

,DeptID,LastName,DeptName
0,NaN,Williams,None
1,35.0,None,Marketing
2,34.0,Robinson,Clerical
3,34.0,Smith,Clerical
4,31.0,Rafferty,Sales
5,33.0,Jones,Engineering
6,33.0,Heisenberg,Engineering


## <font color="red"> Exercise 2 </font>

Determine all the players that have hit more than 50 home runs in a season.  The final table should include the players proper name, as well as the team name.  

**Hint:** You will need join the files listed below.  To get credit for this exercise, use the join `pyspark` join methods presented above.

In [25]:
import pandas as pd
from dfply import *
f1, f2, f3 = ("./data/baseball/core/Batting.csv", 
              "./data/baseball/core/People.csv",
              "./data/baseball/core/Teams.csv")
batting_raw = pd.read_csv(f1)
people_raw = pd.read_csv(f2)
teams_raw = pd.read_csv(f3)
batting = (batting_raw >>
          mutate(id = batting_raw.index))
people = (people_raw >>
         mutate(id = people_raw.index))
teams = (teams_raw >>
        mutate(id = teams_raw.index))


In [28]:
from pyspark.sql import SparkSession
from more_pyspark import get_spark_types, to_pandas

spark = SparkSession.builder.appName('Ops').getOrCreate()

schema1 = get_spark_types(batting_raw, keys=['id'])
schema2 = get_spark_types(people_raw, keys=['id'])
schema3 = get_spark_types(teams_raw, keys = ['id'])

In [31]:
spark = SparkSession.builder.appName('Ops').getOrCreate()
schema_bat = get_spark_types(batting)
batting = spark.createDataFrame(f1, schema=schema1)
people = spark.createDataFrame(f2, schema=schema2)
teams = spark.createDataFrame(f3, schema=schema3)

TypeError: StructType can not accept object '.' in type <class 'str'>

In [9]:
batting.take(5) >> to_pandas

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NA,1,4,0,0,0,...,0,0,0,0,0,None,None,None,None,0
1,addybo01,1871,1,RC1,NA,25,118,30,32,6,...,13,8,1,4,0,None,None,None,None,0
2,allisar01,1871,1,CL1,NA,29,137,28,40,4,...,19,3,1,2,5,None,None,None,None,1
3,allisdo01,1871,1,WS3,NA,27,133,28,44,10,...,27,1,1,0,2,None,None,None,None,0
4,ansonca01,1871,1,RC1,NA,25,120,29,39,11,...,16,6,2,2,1,None,None,None,None,0


In [10]:
people.take(5) >> to_pandas

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,...,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,aardsda01,1981,12,27,USA,CO,Denver,NaN,NaN,NaN,...,Aardsma,David Allan,215,75,R,R,2004-04-05 18:00:00,2015-08-22 19:00:00,aardd001,aardsda01
1,aaronha01,1934,2,5,USA,AL,Mobile,NaN,NaN,NaN,...,Aaron,Henry Louis,180,72,R,R,1954-04-12 18:00:00,1976-10-02 18:00:00,aaroh101,aaronha01
2,aaronto01,1939,8,5,USA,AL,Mobile,1984.0,8.0,16.0,...,Aaron,Tommie Lee,190,75,R,R,1962-04-09 18:00:00,1971-09-25 18:00:00,aarot101,aaronto01
3,aasedo01,1954,9,8,USA,CA,Orange,NaN,NaN,NaN,...,Aase,Donald William,190,75,R,R,1977-07-25 18:00:00,1990-10-02 18:00:00,aased001,aasedo01
4,abadan01,1972,8,25,USA,FL,Palm Beach,NaN,NaN,NaN,...,Abad,Fausto Andres,184,73,L,L,2001-09-09 18:00:00,2006-04-12 19:00:00,abada001,abadan01


In [11]:
teams.take(5) >> to_pandas

,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,...,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
0,1871,NA,BS1,BNA,None,3,31,None,20,10,...,24,0.834,Boston Red Stockings,South End Grounds I,None,103,98,BOS,BS1,BS1
1,1871,NA,CH1,CNA,None,2,28,None,19,9,...,16,0.829,Chicago White Stockings,Union Base-Ball Grounds,None,104,102,CHI,CH1,CH1
2,1871,NA,CL1,CFC,None,8,29,None,10,19,...,15,0.818,Cleveland Forest Citys,National Association Grounds,None,96,100,CLE,CL1,CL1
3,1871,NA,FW1,KEK,None,7,19,None,7,12,...,8,0.803,Fort Wayne Kekiongas,Hamilton Field,None,101,107,KEK,FW1,FW1
4,1871,NA,NY2,NNA,None,5,33,None,16,17,...,14,0.840,New York Mutuals,Union Grounds (Brooklyn),None,90,88,NYU,NY2,NY2


In [12]:
import pyspark.sql.functions as fn
import pyspark.sql.functions as fn
from pyspark.sql.functions import col, isnan
(batting
 .select([batting.yearID, 
         batting.teamID, 
         batting.HR,
         batting.playerID])
 #.groupby(batting.playerID, 
 #          batting.teamID)
 #.agg(fn.sum(pitching_spark.W).alias('total_wins'))
 .where(fn.batting.HRS > 50)
 #.orderBy(pitching_spark.yearID, fn.col('total_wins'), ascending=False)
 .collect()
) >> to_pandas >> head

AttributeError: module 'pyspark.sql.functions' has no attribute 'batting'

## Up Next

Stuff